## [AIBoosted.dev](https://aiboosted.dev)
### A basic Langchain.js chain with prompt template, structured JSON output and Ollama LLMs

In this example, in this example I show you a basic LLM chain, that can review and improve a text. We create a basic LLM chain using LangChain.js. 
We use prompt templates, and ask for structured JSON output.

**To run this notebook locally, set up your own local Jupyter dev environment: [Create an AI prototyping environment using Jupyter Lab IDE with Typescript, LangChain.js and Ollama for rapid AI prototyping](https://www.aiboosted.dev/p/install-jupyter-lab-deno-typescript-prototyping)**

In [1]:
// auto-load .env
import "https://deno.land/std@0.215.0/dotenv/load.ts";

[Module: null prototype] {  }

In [2]:
import { z } from "npm:zod";
import { zodToJsonSchema } from "npm:zod-to-json-schema";
import { OllamaFunctions } from "npm:@langchain/community@~0.2/experimental/chat_models/ollama_functions";
import { PromptTemplate } from "npm:@langchain/core@~0.2/prompts";
import { JsonOutputFunctionsParser } from "npm:@langchain/core@~0.2/output_parsers/openai_functions";

In [3]:
/**
 * Processes a given text using a language model to review and correct it based on the provided instruction.
 * 
 * @param instruction - Instruction for the language model on how to process the text.
 * @param inputText - The text that needs to be reviewed and corrected.
 * @returns The reviewed text if successful, otherwise undefined.
 */
async function reviewTextOllama(instruction: string, inputText: string): Promise<string | undefined> {
  // Create a prompt template by combining the instruction and input text
  const prompt = PromptTemplate.fromTemplate(
`{instruction}
---
{inputText}`);

  // Define a schema for the expected output using zod
  const reviewedTextSchema = z.object({
    reviewedText: z.string().describe("The reviewed text.") // Define the structure and description of the reviewed text
  });
  type ReviewedTextSchema = z.infer<typeof reviewedTextSchema>; // Infer the TypeScript type from the zod schema

  // Initialize the language model with specific configuration
  const llm = new OllamaFunctions({
    baseUrl: "http://localhost:11434", // Base URL for the language model server
    model: "codellama:7b-code", // Specify the model to use
    verbose: false, // Disable verbose logging
  }).bind({
    functions: [
      {
        name: "storeResultTool", // Function name used in the language model
        description: "Gets the reviewed text", // Description of the function
        parameters: {
          type: "object", // Define the type of parameters expected by the function
          properties: zodToJsonSchema(reviewedTextSchema), // Convert zod schema to JSON schema
        },
      },
    ],
    function_call: {
      name: "storeResultTool", // Specify the function to be called
    },
  });

  // Create a processing chain: prompt -> language model -> JSON output parser
  const chain = prompt.pipe(llm).pipe(new JsonOutputFunctionsParser());
  
  // Invoke the chain with the instruction and input text
  const response = await chain.invoke({ instruction, inputText });
  
  // Return the reviewed text if available
  return response?.reviewedText;
}

In [ ]:
// Define the instruction and input text for the prompt
const instruction = "Fix the grammar issues in the following text.";
const inputText = "How to stays relevant as the developer in the world of ai?";

// Log the result of the review function
console.log(await reviewTextOllama(instruction, inputText));